<a href="https://colab.research.google.com/github/hc-satoshiwatanabe/AspNetCore.Docs.Samples/blob/main/%E6%A9%9F%E6%A2%B0%E5%99%A8%E5%85%B7%E6%A4%9C%E5%AE%9A%E7%B0%BF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from io import StringIO
import ipywidgets as widgets
from IPython.display import display, clear_output

# ====== CSVデータ ======
data = """空寸（mm）,容量（ml）,1mm当たりの容量
0,2480,14.018
214,5480,13.888
430,8480,14.018
644,11480,14.018
858,14480,13.888
1074,17480,13.888
1110,17980,11.904
1152,18480,11.904
1194,18980,11.904
1236,19480,12.5
1276,19980,12.5
1316,20480,13.157
1354,20980,13.888
1390,21480,13.909
1578,24095,13.888
1794,27095,13.888
2010,30095,13.888
2226,33095,14.018
2440,36095,13.888
2656,39095,13.888
2872,42095,14.285
2886,42295,14.285
2900,42495,14.285
2914,42695,14.285
2928,42895,14.285
2942,43095,12.5
2958,43295,14.285
2972,43495,12.5
2988,43695,14.285
3002,43895,14.285
3016,44095,12.5
3024,44195,12.5
3032,44295,12.5
3040,44395,12.5
3048,44495,12.5
3056,44595,12.5
3064,44695,12.5
3072,44795,10
3082,44895,12.5
3090,44995,12.5
3098,45095,10
3108,45195,10
3118,45295,10
3128,45395,8.333
3140,45495,10
3150,45595,8.333
3162,45695,8.333
3174,45795,7.142
3188,45895,6.25
3204,45995,3.571
3232,46095,0"""

# ====== DataFrame準備 ======
df = pd.read_csv(StringIO(data))
df.columns = [c.strip().replace('）', ')').replace('（', '(') for c in df.columns]
df['空寸(mm)'] = pd.to_numeric(df['空寸(mm)'])
df['容量(ml)'] = pd.to_numeric(df['容量(ml)'])
df['1mm当たりの容量'] = pd.to_numeric(df['1mm当たりの容量'])
df = df.dropna(subset=['空寸(mm)', '容量(ml)'])

# 最大高さ（タンク上端からの空寸の最大値）
H_max = df['空寸(mm)'].max()

# 半径 = sqrt(1mm当たり容量/π)
df['radius'] = np.sqrt(df['1mm当たりの容量'] / np.pi)

# ====== タンク断面メッシュ作成 ======
theta = np.linspace(0, 2*np.pi, 100)
z_smooth = np.linspace(df['空寸(mm)'].min(), df['空寸(mm)'].max(), 400)
radius_interp = np.interp(z_smooth, df['空寸(mm)'], df['radius'])
Theta, Z_raw = np.meshgrid(theta, z_smooth)
R = np.tile(radius_interp, (theta.size, 1)).T
X = R * np.cos(Theta)
Y = R * np.sin(Theta)
# 上端からの空寸 → 底からの高さに変換
Z = H_max - Z_raw

# ====== 水面生成関数 ======
def create_water_surface(volume_ml, thickness=4):
    # 空寸レベル計算
    level = np.interp(volume_ml, df['容量(ml)'], df['空寸(mm)'])
    radius_level = np.interp(level, df['空寸(mm)'], df['radius'])
    # 底からの高さに変換
    level_from_bottom = H_max - level
    Zw = np.linspace(level_from_bottom - thickness/2, level_from_bottom + thickness/2, 2)[:, None] * np.ones((2, theta.size))
    Xw = np.tile(radius_level * np.cos(theta)[None,:], (2,1))
    Yw = np.tile(radius_level * np.sin(theta)[None,:], (2,1))
    return Xw, Yw, Zw

# ====== 描画関数 ======
def plot_tank(volume):
    clear_output(wait=True)
    Xw, Yw, Zw = create_water_surface(volume)
    fig = go.Figure()

    # タンク本体
    fig.add_trace(go.Surface(
        x=X, y=Y, z=Z,
        colorscale='Greys', opacity=0.3, showscale=False
    ))

    # 液面
    fig.add_trace(go.Surface(
        x=Xw, y=Yw, z=Zw,
        colorscale=[[0,'royalblue'],[1,'royalblue']],
        opacity=0.9, showscale=False,
        lighting=dict(ambient=0.9, diffuse=0.8, specular=0.5)
    ))

    fig.update_layout(
        title=f'3Dタンク（液量: {volume:,} ml）',
        scene=dict(
            xaxis_title='X (mm)',
            yaxis_title='Y (mm)',
            zaxis_title='高さ (mm)',
            aspectratio=dict(x=1, y=1, z=2)
        ),
        width=700,
        height=700
    )
    fig.show()

# ====== ウィジェットUI ======
volume_input = widgets.IntText(
    value=20000,
    description='液量 (ml):',
    step=500
)
button = widgets.Button(description="描画", button_style='primary')

def on_button_clicked(b):
    plot_tank(volume_input.value)

button.on_click(on_button_clicked)
display(volume_input, button)
